# surprise 라이브러리 설치

In [5]:
conda install -c conda-forge scikit-surprise

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\sehyu\anaconda3

  added / updated specs:
    - scikit-surprise


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    scikit-surprise-1.1.0      |py37hc8d92b1_1001         608 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         608 KB

The following NEW packages will be INSTALLED:

  scikit-surprise    conda-forge/win-64::scikit-surprise-1.1.0-py37hc8d92b1_1001




scikit-surprise-1.1. | 608 KB    |            |   0% 
scikit-surprise-1.1. | 608 KB    | ########9  |  89% 
scikit-surprise-1.1. | 608 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


# 패키지 불러오기

In [1]:
import sklearn
import surprise
import pandas as pd
import numpy as np

# 저장된 meetings와 moim 데이터 불러오기와서 합침

In [7]:
meetings = pd.read_csv('./meeting_data.csv')
moim = pd.read_csv('./moim_data.csv')
df = pd.concat([meetings, moim])
df

,meeting_id,uid,writer,created_at,updated_at,is_period,meeting_date,period_date,is_class,max_person,...,unit,is_active,like_cnt,view_cnt,score,main_category,tags,title,area1,area2
0,108923,https://www.frip.co.kr/hosts/617,그 아저씨,2020-05-30 20:58:48.363822,2020-05-30 20:58:48.363822,0,2020-05-30 20:58:48.363822,NaN,1,20,...,원,1,11222,0,4.77,0,NaN,[널위한선물] 도예 공방 원데이 / 도자기 그릇 만들기 (예약가능),서울시,성동구
1,133413,https://www.frip.co.kr/hosts/5545,MCJ,2020-05-30 20:58:52.561736,2020-05-30 20:58:52.561736,0,2020-06-03 19:30:00.000000,NaN,1,20,...,원,1,265,0,5.00,0,NaN,BYOB 각자 와인 1병씩 가져오는 MCJ 홈파티,서울시,강남구
2,101774,https://www.frip.co.kr/hosts/746,아이홉 맥주공방 (I'HOP),2020-05-30 20:58:55.192702,2020-05-30 20:58:55.192702,0,2020-05-31 13:00:00.000000,NaN,1,20,...,원,1,9848,0,4.76,0,NaN,[가정의달] 술술 놀면서 수제 맥주 만들기,서울시,송파구
3,133023,https://www.frip.co.kr/hosts/10654,비푸머스,2020-05-30 20:58:58.940685,2020-05-30 20:58:58.940685,0,2020-05-30 20:58:58.940685,NaN,1,20,...,원,1,888,0,4.81,0,NaN,[서울/홍대] 최상급 프랑스 향료로 나만의 향수 만들기 (예약 가능),서울시,마포구
4,115497,https://www.frip.co.kr/hosts/2839,지아네요리공작소,2020-05-30 20:59:02.833314,2020-05-30 20:59:02.833314,0,2020-05-30 20:59:02.833314,NaN,1,20,...,원,1,5674,0,4.89,0,NaN,[망원동] 스콘 2종 원데이클래스 (예약 가능),서울시,마포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,135609,https://www.frip.co.kr/hosts/11305,박초롱,2020-05-30 17:59:18.149290,2020-05-30 17:59:18.149290,0,2020-06-09 19:30:00.000000,NaN,1,20,...,원,1,66,0,0.00,0,NaN,[4회] 프로N잡러와 함께 - 언젠가 프리랜서,서울,마포구
158,134752,https://www.frip.co.kr/hosts/11160,아페리,2020-05-30 17:59:24.630295,2020-05-30 17:59:24.630295,0,2020-06-06 19:00:00.000000,NaN,1,20,...,원,1,118,0,5.00,0,NaN,2030 직장인 소셜 네트워킹 와인 파티,서울시,중구
159,133251,https://www.frip.co.kr/hosts/10766,수오재기,2020-05-30 17:59:31.681293,2020-05-30 17:59:31.681293,0,2020-06-16 00:00:00.000000,NaN,1,20,...,원,1,64,0,4.90,0,NaN,"15일 _ 토닥토닥, 나 칭찬하기",생활은,"일상에서,"
160,128055,https://www.frip.co.kr/hosts/5493,에스프레소,2020-05-30 17:59:38.988290,2020-05-30 17:59:38.988290,0,2020-06-05 19:45:00.000000,NaN,1,20,...,원,1,415,0,4.71,0,NaN,[여자 추가모집] 완벽한 타인,서울시,강남구


In [8]:
meeting_images = pd.read_csv('./meeting_images.csv')
moim_images = pd.read_csv('./moim_images.csv')
images = pd.concat([meeting_images, moim_images])
images

,Unnamed: 0,meeting_id,image_url
0,0.0,108923,None
1,1.0,133413,None
2,2.0,101774,None
3,3.0,133023,None
4,4.0,115497,None
...,...,...,...
157,NaN,135609,None
158,NaN,134752,None
159,NaN,133251,None
160,NaN,128055,None


# 합친 모임/강좌 정보와 이미지 링크 정보를 csv 파일로 저장

In [15]:
df.to_csv('meetings.csv', index=False, encoding='utf-8-sig')
images.to_csv('images.csv', index=False, encoding='utf-8-sig')

# 합침 정보 불러오기

In [16]:
df = pd.read_csv('meetings.csv')
images = pd.read_csv('images.csv')

In [17]:
df.dtypes

meeting_id         int64
uid               object
writer            object
created_at        object
updated_at        object
is_period          int64
meeting_date      object
period_date      float64
is_class           int64
max_person         int64
now_person         int64
content           object
ref_url           object
address           object
fee                int64
unit              object
is_active          int64
like_cnt           int64
view_cnt           int64
score            float64
main_category      int64
tags             float64
title             object
area1             object
area2             object
dtype: object

In [18]:
df.meeting_id = df.meeting_id.astype('int')

In [12]:
images.dtypes

Unnamed: 0    float64
meeting_id      int64
image_url      object
dtype: object